# Baseline models on Meta data

In [507]:
import pandas as pd
from pandas.core.frame import DataFrame
import numpy as np
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.utils import column_or_1d
from typing import Tuple

In [508]:
##TODO: change directory to cwd if run on colab
%cd /content/drive/MyDrive/Colab Notebooks/vogel
%ls

/content/drive/MyDrive/Colab Notebooks/vogel
 Baseline.ipynb
'Copy of Glycomics_UGA4_Day0Day28.xlsx'
'Copy of Metabolomics_UGA4_D0toDay28.xlsx'
'Copy of metabolomics_UGA4_global_median_normalized_sample_only.quantified.txt'
'Copy of metabolomics_UGA4_untargeted_median_normalized_sample_only.quantified.txt'
'Copy of Proteomics_UGA4_Day0.xlsx'
'Copy of Transcriptomics_UGA4_day0_logCPMcounts.xlsx'
 data_inspect.ipynb
'fluvacc metadata - UGA1-5 - 1368 entries with clear vacc status - for figures.txt'
 meta.txt


In [509]:
df = pd.read_csv("fluvacc metadata - UGA1-5 - 1368 entries with clear vacc status - for figures.txt", sep="\t")  ##TODO: put the name of the file here
df.head()

,Cohort,ID,Age,BMI,BMI_category,Gender,Race,Comorbidities,PreVacc_status,PreVacc_status_year,Month_vaccinated,Vaccine_dose,D0_Titer_H1N1,D28_Titer_H1N1,D0_Titer_H3N2,D28_Titer_H3N2,D0_Titer_IBV_Yam,D28_Titer_IBV_Yam,D0_Titer_IBV_Vic,D28_Titer_IBV_Vic,Seroconversion_H1N1,Seroconversion_H3N2,Seroconversion_IBV_Yam,Seroconversion_IBV_Vic,Composite_seroconversion,SC_category_H1N1,SC_category_H3N2,SC_category_IBV_Yam,SC_category_IBV_Vic,Composite_SC_category,Serostatus_H1N1,Serostatus_H3N2,Serostatus_IBV_Yam,Serostatus_IBV_Vic,Num_SeroPos_strains,Baseline_category_Num_SeroPos_strains,Composite_baseline
0,UGA1,A10,25,33.71,Obese,Male,White,No,Prevaccinated,Year_0,Jan.,Standard,160,160,640,640,1280,1280,320,320,0,0.0,0,0.0,0.0,None,None,None,None,None,1,1,1,1,4,High,35.0
1,UGA1,A100,47,33.37,Obese,Female,White,No,Prevaccinated,Year_0,Jan.,Standard,20,40,20,40,80,80,10,160,1,1.0,0,4.0,6.0,Low,Low,None,High,Low,0,0,1,0,1,Low,18.0
2,UGA1,A101,30,24.99,Normal,Male,Non-white,Yes,Naive,Year_3,Jan.,Standard,20,640,20,320,160,1280,40,2560,5,4.0,3,6.0,18.0,High,High,High,High,High,0,0,1,1,2,Low,21.0
3,UGA1,A102,47,25.00,Overweight,Male,White,No,Naive,Year_3,Feb.,Standard,80,160,40,80,640,1280,20,640,1,1.0,1,5.0,8.0,Low,Low,Low,High,High,1,1,1,0,3,High,25.0
4,UGA1,A103,20,22.10,Normal,Female,Non-white,No,Naive,Year_3,Jan.,Standard,40,640,40,320,40,160,40,320,4,3.0,2,3.0,12.0,High,High,High,High,High,1,1,1,1,4,High,21.0


In [510]:
for column in df:
  print(column, sorted(df[column].unique()))

Cohort ['UGA1', 'UGA2', 'UGA3', 'UGA4', 'UGA5']
ID ['A1', 'A10', 'A100', 'A101', 'A102', 'A103', 'A104', 'A105', 'A106', 'A107', 'A108', 'A109', 'A11', 'A110', 'A111', 'A112', 'A113', 'A114', 'A115', 'A116', 'A118', 'A119', 'A120', 'A122', 'A123', 'A124', 'A125', 'A126', 'A127', 'A128', 'A129', 'A13', 'A130', 'A131', 'A132', 'A133', 'A134', 'A135', 'A136', 'A137', 'A138', 'A139', 'A14', 'A140', 'A141', 'A142', 'A143', 'A144', 'A145', 'A146', 'A147', 'A148', 'A149', 'A15', 'A150', 'A151', 'A152', 'A154', 'A155', 'A156', 'A157', 'A158', 'A16', 'A160', 'A161', 'A162', 'A163', 'A165', 'A166', 'A168', 'A169', 'A17', 'A170', 'A171', 'A172', 'A174', 'A175', 'A176', 'A177', 'A178', 'A179', 'A18', 'A180', 'A181', 'A182', 'A184', 'A185', 'A186', 'A188', 'A19', 'A190', 'A191', 'A192', 'A193', 'A194', 'A195', 'A196', 'A197', 'A199', 'A2', 'A20', 'A200', 'A201', 'A203', 'A204', 'A205', 'A206', 'A207', 'A208', 'A209', 'A21', 'A210', 'A211', 'A212', 'A213', 'A214', 'A215', 'A216', 'A217', 'A218', 'A2

# Basic Preprocessing

* Some entries for $\texttt{BMI_category}$ have unnecessary spaces, to fix:

In [511]:
df.loc[df["BMI_category"] == "Normal ", "BMI_category"] = "Normal"
df.loc[df["BMI_category"] == "Obese ", "BMI_category"] = "Obese"
for column in df:
  print(column, sorted(df[column].unique()))

Cohort ['UGA1', 'UGA2', 'UGA3', 'UGA4', 'UGA5']
ID ['A1', 'A10', 'A100', 'A101', 'A102', 'A103', 'A104', 'A105', 'A106', 'A107', 'A108', 'A109', 'A11', 'A110', 'A111', 'A112', 'A113', 'A114', 'A115', 'A116', 'A118', 'A119', 'A120', 'A122', 'A123', 'A124', 'A125', 'A126', 'A127', 'A128', 'A129', 'A13', 'A130', 'A131', 'A132', 'A133', 'A134', 'A135', 'A136', 'A137', 'A138', 'A139', 'A14', 'A140', 'A141', 'A142', 'A143', 'A144', 'A145', 'A146', 'A147', 'A148', 'A149', 'A15', 'A150', 'A151', 'A152', 'A154', 'A155', 'A156', 'A157', 'A158', 'A16', 'A160', 'A161', 'A162', 'A163', 'A165', 'A166', 'A168', 'A169', 'A17', 'A170', 'A171', 'A172', 'A174', 'A175', 'A176', 'A177', 'A178', 'A179', 'A18', 'A180', 'A181', 'A182', 'A184', 'A185', 'A186', 'A188', 'A19', 'A190', 'A191', 'A192', 'A193', 'A194', 'A195', 'A196', 'A197', 'A199', 'A2', 'A20', 'A200', 'A201', 'A203', 'A204', 'A205', 'A206', 'A207', 'A208', 'A209', 'A21', 'A210', 'A211', 'A212', 'A213', 'A214', 'A215', 'A216', 'A217', 'A218', 'A2

* Co-linear Columns
  Some columns are obviously correlated. The following shows perfect colinearity between $\texttt{PreVacc_status}$ and $\texttt{PreVacc_status_year}$:

In [512]:
print(df[df["PreVacc_status"] == "Naive"][["PreVacc_status_year"]].describe())
print(df[df["PreVacc_status"] == "Prevaccinated"][["PreVacc_status_year"]].describe())


       PreVacc_status_year
count                  200
unique                   1
top                 Year_3
freq                   200
       PreVacc_status_year
count                 1168
unique                   1
top                 Year_0
freq                  1168


This means that we can drop one of the two columns:

In [513]:
df = df.drop("PreVacc_status_year", axis=1)
df

,Cohort,ID,Age,BMI,BMI_category,Gender,Race,Comorbidities,PreVacc_status,Month_vaccinated,Vaccine_dose,D0_Titer_H1N1,D28_Titer_H1N1,D0_Titer_H3N2,D28_Titer_H3N2,D0_Titer_IBV_Yam,D28_Titer_IBV_Yam,D0_Titer_IBV_Vic,D28_Titer_IBV_Vic,Seroconversion_H1N1,Seroconversion_H3N2,Seroconversion_IBV_Yam,Seroconversion_IBV_Vic,Composite_seroconversion,SC_category_H1N1,SC_category_H3N2,SC_category_IBV_Yam,SC_category_IBV_Vic,Composite_SC_category,Serostatus_H1N1,Serostatus_H3N2,Serostatus_IBV_Yam,Serostatus_IBV_Vic,Num_SeroPos_strains,Baseline_category_Num_SeroPos_strains,Composite_baseline
0,UGA1,A10,25,33.71,Obese,Male,White,No,Prevaccinated,Jan.,Standard,160,160,640,640,1280,1280,320,320,0,0.0,0,0.0,0.0,None,None,None,None,None,1,1,1,1,4,High,35.00
1,UGA1,A100,47,33.37,Obese,Female,White,No,Prevaccinated,Jan.,Standard,20,40,20,40,80,80,10,160,1,1.0,0,4.0,6.0,Low,Low,None,High,Low,0,0,1,0,1,Low,18.00
2,UGA1,A101,30,24.99,Normal,Male,Non-white,Yes,Naive,Jan.,Standard,20,640,20,320,160,1280,40,2560,5,4.0,3,6.0,18.0,High,High,High,High,High,0,0,1,1,2,Low,21.00
3,UGA1,A102,47,25.00,Overweight,Male,White,No,Naive,Feb.,Standard,80,160,40,80,640,1280,20,640,1,1.0,1,5.0,8.0,Low,Low,Low,High,High,1,1,1,0,3,High,25.00
4,UGA1,A103,20,22.10,Normal,Female,Non-white,No,Naive,Jan.,Standard,40,640,40,320,40,160,40,320,4,3.0,2,3.0,12.0,High,High,High,High,High,1,1,1,1,4,High,21.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1363,UGA5,C90,14,19.77,Normal,Female,White,No,Prevaccinated,Oct.,Standard,80,40,20,40,40,80,20,20,-1,1.0,1,0.0,1.0,None,Low,Low,None,None,1,0,1,0,2,Low,20.29
1364,UGA5,C93,14,18.45,Lean,Male,White,No,Prevaccinated,Oct.,Standard,20,160,40,80,80,320,20,40,3,1.0,2,1.0,7.0,High,Low,High,Low,Low,0,1,1,0,2,Low,20.29
1365,UGA5,C94,15,25.71,Overweight,Male,White,No,Prevaccinated,Nov.,Standard,40,80,10,320,80,320,40,160,1,5.0,2,2.0,10.0,Low,High,High,High,High,1,0,1,1,3,High,20.29
1366,UGA5,C97,17,23.29,Normal,Male,White,No,Prevaccinated,Oct.,Standard,80,160,160,320,160,320,40,160,1,1.0,1,2.0,5.0,Low,Low,Low,High,Low,1,1,1,1,4,High,26.29


* Conserve some categorical columns so that I can do one hot encoding on the other columns whose categories are not quantatively related. These columns are either binary (yes or no), or those that represent a level of intensity.

In [514]:
bmi_cat = df["BMI_category"]
df2 = df.drop("BMI_category", axis=1)
comorbidities = df["Comorbidities"]
df2 = df2.drop("Comorbidities", axis=1)
dose = df["Vaccine_dose"]
df2 = df2.drop("Vaccine_dose", axis=1)
pre_vac = df["PreVacc_status"]
df2 = df2.drop("PreVacc_status", axis=1)
h1n1 = df["SC_category_H1N1"]
df2 = df2.drop("SC_category_H1N1", axis=1)
h3n2 = df["SC_category_H3N2"]
df2 = df2.drop("SC_category_H3N2", axis=1)
yam = df["SC_category_IBV_Yam"]
df2 = df2.drop("SC_category_IBV_Yam", axis=1)
vic = df["SC_category_IBV_Vic"]
df2 = df2.drop("SC_category_IBV_Vic", axis=1)
comp = df["Composite_SC_category"]
df2 = df2.drop("Composite_SC_category", axis=1)
base = df["Baseline_category_Num_SeroPos_strains"]
df2 = df2.drop("Baseline_category_Num_SeroPos_strains", axis=1)
df2


,Cohort,ID,Age,BMI,Gender,Race,Month_vaccinated,D0_Titer_H1N1,D28_Titer_H1N1,D0_Titer_H3N2,D28_Titer_H3N2,D0_Titer_IBV_Yam,D28_Titer_IBV_Yam,D0_Titer_IBV_Vic,D28_Titer_IBV_Vic,Seroconversion_H1N1,Seroconversion_H3N2,Seroconversion_IBV_Yam,Seroconversion_IBV_Vic,Composite_seroconversion,Serostatus_H1N1,Serostatus_H3N2,Serostatus_IBV_Yam,Serostatus_IBV_Vic,Num_SeroPos_strains,Composite_baseline
0,UGA1,A10,25,33.71,Male,White,Jan.,160,160,640,640,1280,1280,320,320,0,0.0,0,0.0,0.0,1,1,1,1,4,35.00
1,UGA1,A100,47,33.37,Female,White,Jan.,20,40,20,40,80,80,10,160,1,1.0,0,4.0,6.0,0,0,1,0,1,18.00
2,UGA1,A101,30,24.99,Male,Non-white,Jan.,20,640,20,320,160,1280,40,2560,5,4.0,3,6.0,18.0,0,0,1,1,2,21.00
3,UGA1,A102,47,25.00,Male,White,Feb.,80,160,40,80,640,1280,20,640,1,1.0,1,5.0,8.0,1,1,1,0,3,25.00
4,UGA1,A103,20,22.10,Female,Non-white,Jan.,40,640,40,320,40,160,40,320,4,3.0,2,3.0,12.0,1,1,1,1,4,21.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1363,UGA5,C90,14,19.77,Female,White,Oct.,80,40,20,40,40,80,20,20,-1,1.0,1,0.0,1.0,1,0,1,0,2,20.29
1364,UGA5,C93,14,18.45,Male,White,Oct.,20,160,40,80,80,320,20,40,3,1.0,2,1.0,7.0,0,1,1,0,2,20.29
1365,UGA5,C94,15,25.71,Male,White,Nov.,40,80,10,320,80,320,40,160,1,5.0,2,2.0,10.0,1,0,1,1,3,20.29
1366,UGA5,C97,17,23.29,Male,White,Oct.,80,160,160,320,160,320,40,160,1,1.0,1,2.0,5.0,1,1,1,1,4,26.29


In [515]:
df2.select_dtypes(exclude=["number"])

,Cohort,ID,Gender,Race,Month_vaccinated
0,UGA1,A10,Male,White,Jan.
1,UGA1,A100,Female,White,Jan.
2,UGA1,A101,Male,Non-white,Jan.
3,UGA1,A102,Male,White,Feb.
4,UGA1,A103,Female,Non-white,Jan.
...,...,...,...,...,...
1363,UGA5,C90,Female,White,Oct.
1364,UGA5,C93,Male,White,Oct.
1365,UGA5,C94,Male,White,Nov.
1366,UGA5,C97,Male,White,Oct.


* Now turn categorical columns into one hot encoding, dropping one role for each column to avoid perfect colinearity. For BMI category, I used label encoder.

In [516]:
for i, column in enumerate(df2.select_dtypes(exclude=["number"]).iloc[:,2:].columns):
  temp = pd.get_dummies(df2[column], drop_first=True, prefix=column)
  df = df.drop(column, axis=1)
  if i != 0 :
    x = x.join(temp)
  else:
    x = temp
x



,Gender_Male,Race_White,Month_vaccinated_Feb.,Month_vaccinated_Jan.,Month_vaccinated_Nov.,Month_vaccinated_Oct.,Month_vaccinated_Sep.
0,1,1,0,1,0,0,0
1,0,1,0,1,0,0,0
2,1,0,0,1,0,0,0
3,1,1,1,0,0,0,0
4,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...
1363,0,1,0,0,0,1,0
1364,1,1,0,0,0,1,0
1365,1,1,0,0,1,0,0
1366,1,1,0,0,0,1,0


* Deal with the rest of categorical columns with label encoder
>*   For each column we want a specific ordering of the values indicating intensity
  *   Need to modify the label encoder



In [517]:
class CustomLabelEncoder(LabelEncoder):
  def fit(self, y, ordering: list):  # take an ordering list as an argument
      y = column_or_1d(y, warn=True)
      assert set(ordering) == set(pd.Series(y).unique())
      self.classes_ = ordering
      return self

ordering_dict = {"BMI_category": (bmi_cat, ["Lean", "Normal", "Overweight", "Obese"]),
                 "Comorbidities": (comorbidities, ["No", "Yes"]),
                 "Vaccine_dose": (dose, ["Standard", "High"]),
                 "PreVacc_status": (pre_vac, ["Naive", "Prevaccinated"]),
                 "SC_category_H1N1": (h1n1, ["None", "Low", "High"]),
                 "SC_category_H3N2": (h3n2, ["None", "Low", "High"]),
                 "SC_category_IBV_Yam": (yam, ["None", "Low", "High"]),
                 "SC_category_IBV_Vic": (vic, ["None", "Low", "High"]),
                 "Composite_SC_category": (comp, ["None", "Low", "High"]),
                 "Baseline_category_Num_SeroPos_strains": (base, ["Low", "High"])}  # mapping from columns to correct ordering


lencoder = CustomLabelEncoder()
for key, value in ordering_dict.items():
  lencoder.fit(value[0], value[1])
  df.loc[:, key] = lencoder.transform(value[0])

df






,Cohort,ID,Age,BMI,BMI_category,Comorbidities,PreVacc_status,Vaccine_dose,D0_Titer_H1N1,D28_Titer_H1N1,D0_Titer_H3N2,D28_Titer_H3N2,D0_Titer_IBV_Yam,D28_Titer_IBV_Yam,D0_Titer_IBV_Vic,D28_Titer_IBV_Vic,Seroconversion_H1N1,Seroconversion_H3N2,Seroconversion_IBV_Yam,Seroconversion_IBV_Vic,Composite_seroconversion,SC_category_H1N1,SC_category_H3N2,SC_category_IBV_Yam,SC_category_IBV_Vic,Composite_SC_category,Serostatus_H1N1,Serostatus_H3N2,Serostatus_IBV_Yam,Serostatus_IBV_Vic,Num_SeroPos_strains,Baseline_category_Num_SeroPos_strains,Composite_baseline
0,UGA1,A10,25,33.71,3,0,1,0,160,160,640,640,1280,1280,320,320,0,0.0,0,0.0,0.0,0,0,0,0,0,1,1,1,1,4,1,35.00
1,UGA1,A100,47,33.37,3,0,1,0,20,40,20,40,80,80,10,160,1,1.0,0,4.0,6.0,1,1,0,2,1,0,0,1,0,1,0,18.00
2,UGA1,A101,30,24.99,1,1,0,0,20,640,20,320,160,1280,40,2560,5,4.0,3,6.0,18.0,2,2,2,2,2,0,0,1,1,2,0,21.00
3,UGA1,A102,47,25.00,2,0,0,0,80,160,40,80,640,1280,20,640,1,1.0,1,5.0,8.0,1,1,1,2,2,1,1,1,0,3,1,25.00
4,UGA1,A103,20,22.10,1,0,0,0,40,640,40,320,40,160,40,320,4,3.0,2,3.0,12.0,2,2,2,2,2,1,1,1,1,4,1,21.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1363,UGA5,C90,14,19.77,1,0,1,0,80,40,20,40,40,80,20,20,-1,1.0,1,0.0,1.0,0,1,1,0,0,1,0,1,0,2,0,20.29
1364,UGA5,C93,14,18.45,0,0,1,0,20,160,40,80,80,320,20,40,3,1.0,2,1.0,7.0,2,1,2,1,1,0,1,1,0,2,0,20.29
1365,UGA5,C94,15,25.71,2,0,1,0,40,80,10,320,80,320,40,160,1,5.0,2,2.0,10.0,1,2,2,2,2,1,0,1,1,3,1,20.29
1366,UGA5,C97,17,23.29,1,0,1,0,80,160,160,320,160,320,40,160,1,1.0,1,2.0,5.0,1,1,1,2,1,1,1,1,1,4,1,26.29


* Combine two parts of the data

In [518]:
new = df.join(x)
new

,Cohort,ID,Age,BMI,BMI_category,Comorbidities,PreVacc_status,Vaccine_dose,D0_Titer_H1N1,D28_Titer_H1N1,D0_Titer_H3N2,D28_Titer_H3N2,D0_Titer_IBV_Yam,D28_Titer_IBV_Yam,D0_Titer_IBV_Vic,D28_Titer_IBV_Vic,Seroconversion_H1N1,Seroconversion_H3N2,Seroconversion_IBV_Yam,Seroconversion_IBV_Vic,Composite_seroconversion,SC_category_H1N1,SC_category_H3N2,SC_category_IBV_Yam,SC_category_IBV_Vic,Composite_SC_category,Serostatus_H1N1,Serostatus_H3N2,Serostatus_IBV_Yam,Serostatus_IBV_Vic,Num_SeroPos_strains,Baseline_category_Num_SeroPos_strains,Composite_baseline,Gender_Male,Race_White,Month_vaccinated_Feb.,Month_vaccinated_Jan.,Month_vaccinated_Nov.,Month_vaccinated_Oct.,Month_vaccinated_Sep.
0,UGA1,A10,25,33.71,3,0,1,0,160,160,640,640,1280,1280,320,320,0,0.0,0,0.0,0.0,0,0,0,0,0,1,1,1,1,4,1,35.00,1,1,0,1,0,0,0
1,UGA1,A100,47,33.37,3,0,1,0,20,40,20,40,80,80,10,160,1,1.0,0,4.0,6.0,1,1,0,2,1,0,0,1,0,1,0,18.00,0,1,0,1,0,0,0
2,UGA1,A101,30,24.99,1,1,0,0,20,640,20,320,160,1280,40,2560,5,4.0,3,6.0,18.0,2,2,2,2,2,0,0,1,1,2,0,21.00,1,0,0,1,0,0,0
3,UGA1,A102,47,25.00,2,0,0,0,80,160,40,80,640,1280,20,640,1,1.0,1,5.0,8.0,1,1,1,2,2,1,1,1,0,3,1,25.00,1,1,1,0,0,0,0
4,UGA1,A103,20,22.10,1,0,0,0,40,640,40,320,40,160,40,320,4,3.0,2,3.0,12.0,2,2,2,2,2,1,1,1,1,4,1,21.00,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1363,UGA5,C90,14,19.77,1,0,1,0,80,40,20,40,40,80,20,20,-1,1.0,1,0.0,1.0,0,1,1,0,0,1,0,1,0,2,0,20.29,0,1,0,0,0,1,0
1364,UGA5,C93,14,18.45,0,0,1,0,20,160,40,80,80,320,20,40,3,1.0,2,1.0,7.0,2,1,2,1,1,0,1,1,0,2,0,20.29,1,1,0,0,0,1,0
1365,UGA5,C94,15,25.71,2,0,1,0,40,80,10,320,80,320,40,160,1,5.0,2,2.0,10.0,1,2,2,2,2,1,0,1,1,3,1,20.29,1,1,0,0,1,0,0
1366,UGA5,C97,17,23.29,1,0,1,0,80,160,160,320,160,320,40,160,1,1.0,1,2.0,5.0,1,1,1,2,1,1,1,1,1,4,1,26.29,1,1,0,0,0,1,0


* Derive two version of the data for future compatibility of omics data

In [519]:
new_UGA4 = new[new.iloc[:,0] == "UGA4"]  # get two versions of the data: UGA4 and full version; drop the ID columns
new_UGA4_strip = new_UGA4.iloc[:,2:]
new_strip = new.iloc[:,2:]

In [520]:
new_strip

,Age,BMI,BMI_category,Comorbidities,PreVacc_status,Vaccine_dose,D0_Titer_H1N1,D28_Titer_H1N1,D0_Titer_H3N2,D28_Titer_H3N2,D0_Titer_IBV_Yam,D28_Titer_IBV_Yam,D0_Titer_IBV_Vic,D28_Titer_IBV_Vic,Seroconversion_H1N1,Seroconversion_H3N2,Seroconversion_IBV_Yam,Seroconversion_IBV_Vic,Composite_seroconversion,SC_category_H1N1,SC_category_H3N2,SC_category_IBV_Yam,SC_category_IBV_Vic,Composite_SC_category,Serostatus_H1N1,Serostatus_H3N2,Serostatus_IBV_Yam,Serostatus_IBV_Vic,Num_SeroPos_strains,Baseline_category_Num_SeroPos_strains,Composite_baseline,Gender_Male,Race_White,Month_vaccinated_Feb.,Month_vaccinated_Jan.,Month_vaccinated_Nov.,Month_vaccinated_Oct.,Month_vaccinated_Sep.
0,25,33.71,3,0,1,0,160,160,640,640,1280,1280,320,320,0,0.0,0,0.0,0.0,0,0,0,0,0,1,1,1,1,4,1,35.00,1,1,0,1,0,0,0
1,47,33.37,3,0,1,0,20,40,20,40,80,80,10,160,1,1.0,0,4.0,6.0,1,1,0,2,1,0,0,1,0,1,0,18.00,0,1,0,1,0,0,0
2,30,24.99,1,1,0,0,20,640,20,320,160,1280,40,2560,5,4.0,3,6.0,18.0,2,2,2,2,2,0,0,1,1,2,0,21.00,1,0,0,1,0,0,0
3,47,25.00,2,0,0,0,80,160,40,80,640,1280,20,640,1,1.0,1,5.0,8.0,1,1,1,2,2,1,1,1,0,3,1,25.00,1,1,1,0,0,0,0
4,20,22.10,1,0,0,0,40,640,40,320,40,160,40,320,4,3.0,2,3.0,12.0,2,2,2,2,2,1,1,1,1,4,1,21.00,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1363,14,19.77,1,0,1,0,80,40,20,40,40,80,20,20,-1,1.0,1,0.0,1.0,0,1,1,0,0,1,0,1,0,2,0,20.29,0,1,0,0,0,1,0
1364,14,18.45,0,0,1,0,20,160,40,80,80,320,20,40,3,1.0,2,1.0,7.0,2,1,2,1,1,0,1,1,0,2,0,20.29,1,1,0,0,0,1,0
1365,15,25.71,2,0,1,0,40,80,10,320,80,320,40,160,1,5.0,2,2.0,10.0,1,2,2,2,2,1,0,1,1,3,1,20.29,1,1,0,0,1,0,0
1366,17,23.29,1,0,1,0,80,160,160,320,160,320,40,160,1,1.0,1,2.0,5.0,1,1,1,2,1,1,1,1,1,4,1,26.29,1,1,0,0,0,1,0


In [521]:
new_UGA4_strip

,Age,BMI,BMI_category,Comorbidities,PreVacc_status,Vaccine_dose,D0_Titer_H1N1,D28_Titer_H1N1,D0_Titer_H3N2,D28_Titer_H3N2,D0_Titer_IBV_Yam,D28_Titer_IBV_Yam,D0_Titer_IBV_Vic,D28_Titer_IBV_Vic,Seroconversion_H1N1,Seroconversion_H3N2,Seroconversion_IBV_Yam,Seroconversion_IBV_Vic,Composite_seroconversion,SC_category_H1N1,SC_category_H3N2,SC_category_IBV_Yam,SC_category_IBV_Vic,Composite_SC_category,Serostatus_H1N1,Serostatus_H3N2,Serostatus_IBV_Yam,Serostatus_IBV_Vic,Num_SeroPos_strains,Baseline_category_Num_SeroPos_strains,Composite_baseline,Gender_Male,Race_White,Month_vaccinated_Feb.,Month_vaccinated_Jan.,Month_vaccinated_Nov.,Month_vaccinated_Oct.,Month_vaccinated_Sep.
597,31,38.13,3,0,1,0,40,80,5,160,160,1280,40,320,1,5.0,3,3.0,12.0,1,2,2,2,2,1,0,1,1,3,1,20.0,0,1,0,0,0,1,0
598,49,33.37,3,0,1,0,5,20,20,160,10,20,20,40,2,3.0,1,1.0,7.0,2,2,1,1,1,0,0,0,0,0,0,14.0,0,1,0,0,0,1,0
599,28,38.93,3,0,1,0,160,160,10,20,640,640,80,80,0,1.0,0,0.0,1.0,0,1,0,0,0,1,0,1,1,3,1,26.0,0,1,0,1,0,0,0
600,25,21.26,1,0,1,0,40,80,40,320,40,80,160,160,1,3.0,1,0.0,5.0,1,2,1,0,1,1,1,1,1,4,1,23.0,0,1,0,0,0,1,0
601,76,25.99,2,1,1,1,5,10,5,10,10,10,20,40,1,1.0,0,1.0,3.0,1,1,0,1,0,0,0,0,0,0,0,12.0,1,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1032,14,21.76,1,0,1,0,20,80,80,640,80,80,160,160,2,3.0,0,0.0,5.0,2,2,0,0,1,0,1,1,1,3,1,24.0,0,1,0,0,0,1,0
1033,13,19.20,1,0,1,0,320,320,5,20,40,160,40,80,0,2.0,2,1.0,5.0,0,2,2,1,1,1,0,1,1,3,1,21.0,0,1,0,0,0,1,0
1034,13,18.46,0,0,1,0,160,160,40,40,40,160,5,20,0,0.0,2,2.0,4.0,0,0,2,2,1,1,1,1,0,3,1,20.0,1,1,0,0,0,1,0
1035,16,23.24,1,0,1,0,320,320,80,160,160,160,160,80,0,1.0,0,-1.0,0.0,0,1,0,0,0,1,1,1,1,4,1,29.0,1,1,0,0,1,0,0


# Input/Output selector

* Select which columns to be the input. Refer to the columns as their original names.

* Choose output type $\texttt{tuple(type: str, antigen: str)}$

For type:
 * Seroconversion: regression
 * D28_Titer: regression
 * SC_category: classification
 * Serostatus*: classification 

For antigen:
 * H1N1
 * H3N2
 * IBV_Yam
 * IBV_Vic
 * Composite

\* when antigen is composite, output is $\texttt{high}$ or $\texttt{low}$ for Seropositive strains.



In [522]:
def io_selector(data: DataFrame, 
                input_columns: list = None, 
                output_type: str = None, 
                output_antigen: str = None) -> Tuple[np.ndarray, ...]:
  
  default_input_columns = ["Age", "BMI", "Gender_Male", "Race_White", "Comorbidities", 
                           "PreVacc_status", "Month_vaccinated_Jan.", "Month_vaccinated_Feb.",
                           "Month_vaccinated_Sep.", "Month_vaccinated_Oct.",
                           "Month_vaccinated_Nov.", "Vaccine_dose",
                           "D0_Titer_H1N1", "D0_Titer_H3N2", "D0_Titer_IBV_Yam", 
                           "D0_Titer_IBV_Vic"]
  default_output = ["Composite_seroconversion"]

  name_mapping = {"Gender": ["Gender_Male"], "Race": ["Race_White"], 
                  "Month_vaccinated": ["Month_vaccinated_Jan.",
                                       "Month_vaccinated_Feb.",
                                       "Month_vaccinated_Sep.", 
                                       "Month_vaccinated_Oct.", 
                                       "Month_vaccinated_Nov."]}
  if input_columns is None: 
    input_columns = default_input_columns
  else:
    temp = []
    for item in input_columns:
      if item in name_mapping:
        temp.extend(name_mapping[item])
      else:
        temp.append(item)
    input_columns = temp
  if output_antigen is None:
    output = default_output
  else:
    assert "_".join([output_antigen, output_type]) in data.columns or \
   "_".join([output_type, output_antigen]) in data.columns or \
   (output_type == "Serostatus" and output_antigen == "Composite")
    if output_antigen == "Composite":
      if output_type == "Serostatus":
        output = "Baseline_category_Num_SeroPos_strains"
      else:
        output = "_".join([output_antigen, output_type])
    else:
      output = "_".join([output_type, output_antigen])
  x = data[input_columns]
  y = data[output]
  return (x, y)



Test to see whether it works

In [523]:
io_selector(new_strip)

(      Age    BMI  ...  D0_Titer_IBV_Yam  D0_Titer_IBV_Vic
 0      25  33.71  ...              1280               320
 1      47  33.37  ...                80                10
 2      30  24.99  ...               160                40
 3      47  25.00  ...               640                20
 4      20  22.10  ...                40                40
 ...   ...    ...  ...               ...               ...
 1363   14  19.77  ...                40                20
 1364   14  18.45  ...                80                20
 1365   15  25.71  ...                80                40
 1366   17  23.29  ...               160                40
 1367   16  27.45  ...                20                20
 
 [1368 rows x 16 columns],       Composite_seroconversion
 0                          0.0
 1                          6.0
 2                         18.0
 3                          8.0
 4                         12.0
 ...                        ...
 1363                       1.0
 1364   

# Setting up models
